# Processing mobility data

- Input: the raw mobility data from Moribus (Chicago, Miami, and Boston) e.g. 01_raw/boston_stays_from_esteban.pickle
- Outputs: boston_stays, chicago_stays, and Miami_stays.

Added time of day, day of week, and other information to the data set. An important change: I replaced the census block group indices by the census tract indices. It is because the predictive performance at the CBG levels is not good.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime 
from dateutil import tz
import gzip
import time
import pickle



Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
# read the boston, miami and chicago data
# it takes several minutes to read the files...
with open("../../data/01_raw/boston_stays_from_esteban.pickle", 'rb') as f:
    df_boston = pickle.load(f) # df_raw is Boston.

df_chicago = pd.read_csv('../../data/01_raw/stays2_16980.csv.gz', compression='gzip')

df_miami = pd.read_csv('../../data/01_raw/stays2_33100.csv.gz', compression='gzip')


In [3]:
# turn them to dic
df_dic = {}
df_dic['boston'] = df_boston
df_dic['chicago'] = df_chicago
df_dic['miami'] = df_miami


In [4]:
df_boston

,user,duration,ini_dat,lon_medoid,lat_medoid,GEOID,home_lon_med,home_lat_med,GEOID_home,quant,fsq_id,cat,distPOI,disthome
0,0000ff45f7f170db960e4e601167975f7559c5be147d69...,388,1.481736e+09,-71.258500,42.359965,250173684002,-71.284748,42.558568,250173164003,3,5390bb9c498e21833a58c208,Office,0.001016,22.188364
1,000277100d5593fec35a151e228f6a485210a3fa87cda7...,600,1.478995e+09,-71.149170,42.332710,250214011003,-71.175020,42.300842,250173739004,4,4e5e3cb31838f7255272d5e6,Arts & Entertainment,0.001723,4.132097
2,000277100d5593fec35a151e228f6a485210a3fa87cda7...,4086,1.481411e+09,-71.151375,42.336340,250250005022,-71.175020,42.300842,250173739004,4,4e0b572db61cf2466a0f3c6e,Pool,0.000166,4.399966
3,000277100d5593fec35a151e228f6a485210a3fa87cda7...,1211,1.481563e+09,-71.207125,42.349600,250173734002,-71.175020,42.300842,250173739004,4,51e1752f498ea376f1942680,Jewelry,0.000149,6.029946
4,000277100d5593fec35a151e228f6a485210a3fa87cda7...,621,1.482095e+09,-71.175305,42.318550,250173739001,-71.175020,42.300842,250173739004,4,52edbf4f498e284de9956efb,Supermarket,0.000012,1.969160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12106757,fffe58b2fb58c5940b85ba55b07eb4d1d11b36185a4afb...,10297,1.486467e+09,-70.779474,43.073728,330150693002,-71.120296,42.977653,330150540002,2,4bd5f42ecfa7b713a8bf26da,Diner,0.000526,29.693304
12106758,fffe58b2fb58c5940b85ba55b07eb4d1d11b36185a4afb...,2946,1.486508e+09,-71.164300,43.039123,330150550021,-71.120296,42.977653,330150540002,2,4f9ffdd4e4b03a4a3f829164,Deli / Bodega,0.000105,7.715061
12106759,fffe58b2fb58c5940b85ba55b07eb4d1d11b36185a4afb...,797,1.486661e+09,-71.074862,43.024131,330150590001,-71.120296,42.977653,330150540002,2,4d642a848018548105694768,Pharmacy,0.000487,6.353051
12106760,fffe58b2fb58c5940b85ba55b07eb4d1d11b36185a4afb...,2215,1.486828e+09,-70.883073,42.891006,330150630011,-71.120296,42.977653,330150540002,2,4b7866c0f964a52075cb2ee3,Warehouse Store,0.000262,21.582175


In [6]:
# print the three shapes
print(df_dic['boston'].shape)
print(df_dic['chicago'].shape)
print(df_dic['miami'].shape)


(12053805, 14)
(67509807, 14)
(46219015, 14)


In [7]:
# Check the number of GEOIDs (census block groups)
print(len(np.unique(df_dic['boston'].GEOID)))
print(len(np.unique(df_dic['boston'].GEOID_home)))

3115
3196


In [8]:
# Check the data info
print(df_dic['boston'].shape)
print(df_dic['boston'].columns)

(12053805, 14)
Index(['user', 'duration', 'ini_dat', 'lon_medoid', 'lat_medoid', 'GEOID',
       'home_lon_med', 'home_lat_med', 'GEOID_home', 'quant', 'fsq_id', 'cat',
       'distPOI', 'disthome'],
      dtype='object')


# Process stay data

In [9]:
# Need to process the ini_dat to get: hour of day, day of week, weekday vs. weekend.
for key_ in df_dic.keys():
    # Wait, the Chicago time is off by one hour...
    print(key_)
    print(np.min(df_dic[key_]['ini_dat']), np.max(df_dic[key_]['ini_dat']))
    print("Starting time of the data set:", datetime.fromtimestamp(int(np.min(df_dic[key_]['ini_dat']))).strftime('%Y-%m-%d %H:%M:%S'))
    print("Ending time of the data set:",datetime.fromtimestamp(int(np.max(df_dic[key_]['ini_dat']))).strftime('%Y-%m-%d %H:%M:%S'))

    # Use a baseline unix time to start 
    baseline_unix_time = 1474862400 # Mon Sep 26 2016 00:00:00 GMT-0400 (Eastern Daylight Time)
    # one unit in unix time is one second. 
    # 60 seconds, 60 min, 24 hours, 7 days. 

    # get time info
    minutes = (df_dic[key_]['ini_dat'] - baseline_unix_time)/60
    hours = (df_dic[key_]['ini_dat'] - baseline_unix_time)/3600
    days = (df_dic[key_]['ini_dat'] - baseline_unix_time)/(3600*24)

    # hour of day, day of week, etc.
    hour_of_day = (hours.astype(int) + 1)%24 # 0~23 hours
    day_of_week = (days.astype(int) + 1)%7 # 0~6 days; 0: Sunday. 6: Saturday
    weekday = day_of_week.isin([1,2,3,4,5]) # true/false
    weekend = day_of_week.isin([6,0]) # true/false

    # old col names
    old_col_names = list(df_dic[key_].columns)

    # augment time info
    df_dic[key_] = pd.concat([df_dic[key_], hour_of_day, day_of_week, weekday, weekend], axis = 1)
    df_dic[key_].columns = old_col_names + ['hour_of_day', 'day_of_week', 'weekday', 'weekend']
    


boston
1475210333.0 1491091786.0
Starting time of the data set: 2016-09-30 00:38:53
Ending time of the data set: 2017-04-01 20:09:46
chicago
1475212313.0 1491095383.0
Starting time of the data set: 2016-09-30 01:11:53
Ending time of the data set: 2017-04-01 21:09:43
miami
1475208013.0 1491172174.0
Starting time of the data set: 2016-09-30 00:00:13
Ending time of the data set: 2017-04-02 18:29:34


In [10]:
df_dic['chicago']

,user,duration,ini_dat,lon_medoid,lat_medoid,GEOID,home_lon_med,home_lat_med,GEOID_home,quant,fsq_id,cat,distPOI,disthome,hour_of_day,day_of_week,weekday,weekend
0,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,3409,1.481031e+09,-88.106172,42.027176,170318047103,-88.082159,42.020098,170318048102,4,4bbdcbc08ec3d13add6b1c28,High School,124.189172,2133.964273,10,2,True,False
1,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,2301,1.481041e+09,-88.105845,42.027447,170318047103,-88.082159,42.020098,170318048102,4,5137ba88e4b0828b63bc822f,Office,154.497903,2120.332971,13,2,True,False
2,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,1956,1.481049e+09,-88.106938,42.027169,170318047103,-88.082159,42.020098,170318048102,4,4bbdcbc08ec3d13add6b1c28,High School,48.071712,2192.620204,15,2,True,False
3,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,805,1.481053e+09,-88.107650,42.027230,170318047103,-88.082159,42.020098,170318048102,4,4bbdcbc08ec3d13add6b1c28,High School,23.998529,2250.039029,16,2,True,False
4,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,636,1.481068e+09,-88.082190,42.019920,170318048102,-88.082159,42.020098,170318048102,4,4c607423cd522d7f502dce3f,Elementary School,199.718644,19.916588,20,2,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67509802,ffffdea33b50dc7b779f0882c651630287c2d3c5f50eb0...,2918,1.490262e+09,-88.219202,42.584389,550590028004,-88.218925,42.584764,550590028004,3,4e08bd96d22d658532ad9559,Café,117.680155,47.444643,6,4,True,False
67509803,ffffdea33b50dc7b779f0882c651630287c2d3c5f50eb0...,438,1.490322e+09,-88.219220,42.584395,550590028004,-88.218925,42.584764,550590028004,3,4e08bd96d22d658532ad9559,Café,118.763507,47.561429,23,4,True,False
67509804,ffffdea33b50dc7b779f0882c651630287c2d3c5f50eb0...,914,1.490324e+09,-88.219148,42.584360,550590028004,-88.218925,42.584764,550590028004,3,4e08bd96d22d658532ad9559,Café,113.458272,48.454940,23,4,True,False
67509805,ffffdea33b50dc7b779f0882c651630287c2d3c5f50eb0...,6982,1.490325e+09,-88.219161,42.584327,550590028004,-88.218925,42.584764,550590028004,3,4e08bd96d22d658532ad9559,Café,110.615874,52.256121,0,4,True,False


In [11]:
for key_ in df_dic.keys():
    # change GEOIDs to string
    df_dic[key_].GEOID_home = df_dic[key_].GEOID_home.astype('str')
    df_dic[key_].GEOID = df_dic[key_].GEOID.astype('str')

    # add the df CTs.
    # GEOID_ct_list = []
    df_dic[key_]['GEOID'] = df_dic[key_].GEOID.map(lambda x: x[:11])
    df_dic[key_]['GEOID_home'] = df_dic[key_].GEOID_home.map(lambda x: x[:11])


In [12]:
# Quite slow & useless block... 
# current number of census tracts. 983, 988, 1002.
# fast.
print("Boston")
print("Number of activity census tracts: ", len(np.unique(df_dic['boston']['GEOID'])))
print("Number of home census tracts: ", len(np.unique(df_dic['boston']['GEOID_home'])))
print("Total number of census tracts: ", len(set(np.unique(df_dic['boston']['GEOID'])).union(set(np.unique(df_dic['boston']['GEOID_home']))))) 

# number of census tracts. 2206, 
# slow...
print("Chicago")
print("Number of activity census tracts: ", len(np.unique(df_dic['chicago']['GEOID'])))
print("Number of home census tracts: ", len(np.unique(df_dic['chicago']['GEOID_home'])))
print("Total number of census tracts: ", len(set(np.unique(df_dic['chicago']['GEOID'])).union(set(np.unique(df_dic['chicago']['GEOID_home']))))) 

# number of census tracts. 983, 988, 
# slow...
print("Miami")
print("Number of activity census tracts: ", len(np.unique(df_dic['miami']['GEOID'])))
print("Number of home census tracts: ", len(np.unique(df_dic['miami']['GEOID_home'])))
print("Total number of census tracts: ", len(set(np.unique(df_dic['miami']['GEOID'])).union(set(np.unique(df_dic['miami']['GEOID_home'])))))


Boston
Number of activity census tracts:  983
Number of home census tracts:  988
Total number of census tracts:  1002
Chicago
Number of activity census tracts:  2206
Number of home census tracts:  2197


KeyboardInterrupt: 

In [13]:
df_dic['boston']

,user,duration,ini_dat,lon_medoid,lat_medoid,GEOID,home_lon_med,home_lat_med,GEOID_home,quant,fsq_id,cat,distPOI,disthome,hour_of_day,day_of_week,weekday,weekend
0,0000ff45f7f170db960e4e601167975f7559c5be147d69...,388,1.481736e+09,-71.258500,42.359965,25017368400,-71.284748,42.558568,25017316400,3,5390bb9c498e21833a58c208,Office,0.001016,22.188364,14,3,True,False
1,000277100d5593fec35a151e228f6a485210a3fa87cda7...,600,1.478995e+09,-71.149170,42.332710,25021401100,-71.175020,42.300842,25017373900,4,4e5e3cb31838f7255272d5e6,Arts & Entertainment,0.001723,4.132097,20,6,False,True
2,000277100d5593fec35a151e228f6a485210a3fa87cda7...,4086,1.481411e+09,-71.151375,42.336340,25025000502,-71.175020,42.300842,25017373900,4,4e0b572db61cf2466a0f3c6e,Pool,0.000166,4.399966,20,6,False,True
3,000277100d5593fec35a151e228f6a485210a3fa87cda7...,1211,1.481563e+09,-71.207125,42.349600,25017373400,-71.175020,42.300842,25017373900,4,51e1752f498ea376f1942680,Jewelry,0.000149,6.029946,14,1,True,False
4,000277100d5593fec35a151e228f6a485210a3fa87cda7...,621,1.482095e+09,-71.175305,42.318550,25017373900,-71.175020,42.300842,25017373900,4,52edbf4f498e284de9956efb,Supermarket,0.000012,1.969160,17,0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12106757,fffe58b2fb58c5940b85ba55b07eb4d1d11b36185a4afb...,10297,1.486467e+09,-70.779474,43.073728,33015069300,-71.120296,42.977653,33015054000,2,4bd5f42ecfa7b713a8bf26da,Diner,0.000526,29.693304,8,2,True,False
12106758,fffe58b2fb58c5940b85ba55b07eb4d1d11b36185a4afb...,2946,1.486508e+09,-71.164300,43.039123,33015055002,-71.120296,42.977653,33015054000,2,4f9ffdd4e4b03a4a3f829164,Deli / Bodega,0.000105,7.715061,19,2,True,False
12106759,fffe58b2fb58c5940b85ba55b07eb4d1d11b36185a4afb...,797,1.486661e+09,-71.074862,43.024131,33015059000,-71.120296,42.977653,33015054000,2,4d642a848018548105694768,Pharmacy,0.000487,6.353051,14,4,True,False
12106760,fffe58b2fb58c5940b85ba55b07eb4d1d11b36185a4afb...,2215,1.486828e+09,-70.883073,42.891006,33015063001,-71.120296,42.977653,33015054000,2,4b7866c0f964a52075cb2ee3,Warehouse Store,0.000262,21.582175,12,6,False,True


In [14]:
df_dic['chicago']

,user,duration,ini_dat,lon_medoid,lat_medoid,GEOID,home_lon_med,home_lat_med,GEOID_home,quant,fsq_id,cat,distPOI,disthome,hour_of_day,day_of_week,weekday,weekend
0,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,3409,1.481031e+09,-88.106172,42.027176,17031804710,-88.082159,42.020098,17031804810,4,4bbdcbc08ec3d13add6b1c28,High School,124.189172,2133.964273,10,2,True,False
1,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,2301,1.481041e+09,-88.105845,42.027447,17031804710,-88.082159,42.020098,17031804810,4,5137ba88e4b0828b63bc822f,Office,154.497903,2120.332971,13,2,True,False
2,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,1956,1.481049e+09,-88.106938,42.027169,17031804710,-88.082159,42.020098,17031804810,4,4bbdcbc08ec3d13add6b1c28,High School,48.071712,2192.620204,15,2,True,False
3,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,805,1.481053e+09,-88.107650,42.027230,17031804710,-88.082159,42.020098,17031804810,4,4bbdcbc08ec3d13add6b1c28,High School,23.998529,2250.039029,16,2,True,False
4,00002717f96c7e06c736c1ec32476c93412d2de9f8fa5a...,636,1.481068e+09,-88.082190,42.019920,17031804810,-88.082159,42.020098,17031804810,4,4c607423cd522d7f502dce3f,Elementary School,199.718644,19.916588,20,2,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67509802,ffffdea33b50dc7b779f0882c651630287c2d3c5f50eb0...,2918,1.490262e+09,-88.219202,42.584389,55059002800,-88.218925,42.584764,55059002800,3,4e08bd96d22d658532ad9559,Café,117.680155,47.444643,6,4,True,False
67509803,ffffdea33b50dc7b779f0882c651630287c2d3c5f50eb0...,438,1.490322e+09,-88.219220,42.584395,55059002800,-88.218925,42.584764,55059002800,3,4e08bd96d22d658532ad9559,Café,118.763507,47.561429,23,4,True,False
67509804,ffffdea33b50dc7b779f0882c651630287c2d3c5f50eb0...,914,1.490324e+09,-88.219148,42.584360,55059002800,-88.218925,42.584764,55059002800,3,4e08bd96d22d658532ad9559,Café,113.458272,48.454940,23,4,True,False
67509805,ffffdea33b50dc7b779f0882c651630287c2d3c5f50eb0...,6982,1.490325e+09,-88.219161,42.584327,55059002800,-88.218925,42.584764,55059002800,3,4e08bd96d22d658532ad9559,Café,110.615874,52.256121,0,4,True,False


In [15]:
df_dic['miami']

,user,duration,ini_dat,lon_medoid,lat_medoid,GEOID,home_lon_med,home_lat_med,GEOID_home,quant,fsq_id,cat,distPOI,disthome,hour_of_day,day_of_week,weekday,weekend
0,0000b5e140c59d0f28436cf9e9d37ab9932544a820571a...,960,1.475522e+09,-80.208863,26.222665,12011020501,-80.208863,26.222631,12011020501,2,50326e2ee4b04de6f50f4347,Caribbean,122.061867,3.697231,16,1,True,False
1,0000b5e140c59d0f28436cf9e9d37ab9932544a820571a...,2692,1.475523e+09,-80.209081,26.222506,12011020501,-80.208863,26.222631,12011020501,2,524865c611d27142e64d892b,Park,121.029196,25.830563,16,1,True,False
2,0000b5e140c59d0f28436cf9e9d37ab9932544a820571a...,2832,1.475526e+09,-80.208885,26.222509,12011020501,-80.208863,26.222631,12011020501,2,524865c611d27142e64d892b,Park,113.094925,13.749946,17,1,True,False
3,0000b5e140c59d0f28436cf9e9d37ab9932544a820571a...,6861,1.475645e+09,-80.208885,26.222616,12011020501,-80.208863,26.222631,12011020501,2,50326e2ee4b04de6f50f4347,Caribbean,122.560525,2.764433,2,3,True,False
4,0000b5e140c59d0f28436cf9e9d37ab9932544a820571a...,1943,1.475652e+09,-80.208907,26.222616,12011020501,-80.208863,26.222631,12011020501,2,524865c611d27142e64d892b,Park,123.870468,4.672091,4,3,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46219010,ffffb1a5fa058a91c8db0c4241fadc7667dcefc27b1449...,651,1.490811e+09,-80.230065,25.866730,12086000408,-80.237455,25.849600,12086001003,1,4d710029766da143ad2157ad,Trade School,48.817487,2043.275486,15,3,True,False
46219011,ffffb1a5fa058a91c8db0c4241fadc7667dcefc27b1449...,904,1.490920e+09,-80.264373,25.954238,12086010009,-80.237455,25.849600,12086001003,1,4e29e7d9aeb745b1fab29913,Spiritual,19.432149,11942.669400,21,4,True,False
46219012,ffffb1a5fa058a91c8db0c4241fadc7667dcefc27b1449...,2157,1.490921e+09,-80.264495,25.954195,12086010009,-80.237455,25.849600,12086001003,1,4e29e7d9aeb745b1fab29913,Spiritual,31.064430,11940.747742,21,4,True,False
46219013,ffffb1a5fa058a91c8db0c4241fadc7667dcefc27b1449...,2523,1.490977e+09,-80.228517,25.866778,12086000408,-80.237455,25.849600,12086001003,1,4e1b71c3d16498449dc25df4,School,138.791219,2109.173114,13,5,True,False


# Save

In [16]:
with open("../../data/02_intermediate/boston_stays.pickle", 'wb') as f:
    pickle.dump(df_dic['boston'], f)


In [17]:
with open("../../data/02_intermediate/chicago_stays.pickle", 'wb') as f:
    pickle.dump(df_dic['chicago'], f)


In [18]:
with open("../../data/02_intermediate/miami_stays.pickle", 'wb') as f:
    pickle.dump(df_dic['miami'], f)
